In [1]:
from pathlib import Path
paths = [str(x) for x in Path('/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/').glob('**/*.txt')]

In [2]:
len(paths)

158

In [3]:
paths

['/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_9100000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_5500000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_8500000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_8400000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_6600000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_7600000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_14300000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_4700000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dat

In [4]:
from transformers import DistilBertTokenizerFast, RobertaTokenizerFast, RobertaTokenizer
tokenizer = RobertaTokenizer(vocab_file='/userdirs/piyumal/roberta_sinhala/Roberta_tokenizer_52000/vocab.json',
                             merges_file='/userdirs/piyumal/roberta_sinhala/Roberta_tokenizer_52000/merges.txt', max_len=512)

In [5]:
tokens = tokenizer.encode("ඉතිං බ්ලොග් පිටුවත් ලිව්වා නම් හොදයි කියලා හිතුනා.")

In [6]:
tokenizer.decode(tokens)

'<s>ඉතිං බ්ලොග් පිටුවත් ලිව්වා නම් හොදයි කියලා හිතුනා.</s>'

In [7]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [8]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=1,
)

In [9]:
from transformers import RobertaForMaskedLM

# model = RobertaForMaskedLM(config=config)
model = RobertaForMaskedLM.from_pretrained('./Roberta_models_tokenizer_52000_12_layer/12_layer_model_0_157')

# model = RobertaForMaskedLM.from_pretrained('/userdirs/piyumal/roberta_sinhala/ROberta_models_2/6_layer_model_0_6')

In [10]:
model.num_parameters()
# => 84 million parameters

126031648

In [11]:
from transformers import LineByLineTextDataset,TextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import gc
import pickle

epochs = 1
for epoch in range(epochs):
    for j, path in enumerate(paths):
        if j <=68:
            continue
        dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path=path, block_size =512)
        data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
        training_args = TrainingArguments(
                            output_dir="./Roberta_models_tokenizer_52000_12_layer",
                            overwrite_output_dir=True,
                            num_train_epochs=1,
                            per_gpu_train_batch_size=16,
                            save_steps=1000,
                            save_total_limit=2,
                            prediction_loss_only=True,
                            logging_steps = 100)
        trainer = Trainer(
                    model=model,
                    args=training_args,
                    data_collator=data_collator,
                    train_dataset=dataset)
        trainer.train()
        trainer.save_model(f'./Roberta_models_tokenizer_52000_12_layer/12_layer_model_{epoch+1}_{j}')
        with open(f'./Roberta_models_tokenizer_52000_12_layer/loss_values/loss_value_file_{epoch+1}_{j}', 'wb') as fp:
                pickle.dump(trainer.state.log_history, fp)


        

/userdirs/piyumal/roberta_sinhala/rpt_env/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 100000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6250


Step,Training Loss
100,0.736700
200,0.734100
300,0.769200
400,0.782200
500,0.770500
600,0.734500
700,0.762900
800,0.781400
900,0.756600
1000,0.831000


Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000/config.json
Model weights saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [Roberta_models_tokenizer_52000_12_layer/checkpoint-5000] due to args.save_total_limit
Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000/config.json
Model weights saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [Roberta_models_tokenizer_52000_12_layer/checkpoint-6000] due to args.save_total_limit
Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-3000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-3000/config.json
Model weights saved in ./Roberta_models_to

Step,Training Loss
100,0.716900
200,0.760700
300,0.767700
400,0.732400
500,0.765800
600,0.745700
700,0.795500
800,0.771600
900,0.775000
1000,0.761900


Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000/config.json
Model weights saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [Roberta_models_tokenizer_52000_12_layer/checkpoint-5000] due to args.save_total_limit
Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000/config.json
Model weights saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [Roberta_models_tokenizer_52000_12_layer/checkpoint-6000] due to args.save_total_limit
Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-3000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-3000/config.json
Model weights saved in ./Roberta_models_to

...

Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000/config.json
Model weights saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [Roberta_models_tokenizer_52000_12_layer/checkpoint-5000] due to args.save_total_limit
Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000/config.json
Model weights saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [Roberta_models_tokenizer_52000_12_layer/checkpoint-6000] due to args.save_total_limit
Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-3000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-3000/config.json
Model weights saved in ./Roberta_models_to

Step,Training Loss
100,0.679100
200,0.709000
300,0.740100
400,0.761700
500,0.730300
600,0.715000
700,0.759700
800,0.725600
900,0.750900
1000,0.750300


Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000/config.json
Model weights saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [Roberta_models_tokenizer_52000_12_layer/checkpoint-5000] due to args.save_total_limit
Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000/config.json
Model weights saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [Roberta_models_tokenizer_52000_12_layer/checkpoint-6000] due to args.save_total_limit
Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-3000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-3000/config.json
Model weights saved in ./Roberta_models_to

Step,Training Loss
100,0.646000
200,0.660700
300,0.691200
400,0.751800
500,0.745000
600,0.785300
700,0.769700
800,0.728400
900,0.720700
1000,0.751500


Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000/config.json
Model weights saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [Roberta_models_tokenizer_52000_12_layer/checkpoint-5000] due to args.save_total_limit
Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000/config.json
Model weights saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [Roberta_models_tokenizer_52000_12_layer/checkpoint-6000] due to args.save_total_limit
Saving model checkpoint to ./Roberta_models_tokenizer_52000_12_layer/checkpoint-3000
Configuration saved in ./Roberta_models_tokenizer_52000_12_layer/checkpoint-3000/config.json
Model weights saved in ./Roberta_models_to

In [12]:
trainer.state.log_history[-1]['epoch']

1.0